Author: Charlie Moffett, NYU CUSP, 2017

# Time Series Clustering Exercise

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import os
import sys
import zipfile 

import pylab as pl
%pylab inline

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GMM
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

from scipy.spatial.distance import cdist, pdist

Populating the interactive namespace from numpy and matplotlib


__Download the census bureau business data for all years 1993-2014.__

In [2]:
# move CBB data to PUIdata folder
PUIdata = os.getenv('PUIDATA')

for i in range(94,100):
     os.system('mv zbp' + str(i) + 'totals.zip ' + PUIdata)


for i in range(0,15):
     os.system('mv zbp' + str(i).zfill(2) + 'totals.zip ' + PUIdata)

In [3]:
# unzip NYC zipcode shapefiles 
if not os.path.isdir(PUIdata + '/ZIP_CODES'):
    print('Download and unzip file...')
    os.system('wget https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip')
    os.system('mv application%2Fzip ' + PUIdata)
    os.system('unzip ' + PUIdata + '/application%2Fzip -d ' + PUIdata + '/ZIP_CODES')

if os.path.isdir(PUIdata + '/ZIP_CODES'):
    for file in os.listdir(PUIdata + '/ZIP_CODES'):
        if file.endswith('.shp'):
            print('Shapefile in place..')
            shpfile = file

Shapefile in place..


In [4]:
import pandas as pd
import zipfile 
zf = zipfile.ZipFile(fname)
df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))

NameError: name 'fname' is not defined

In [9]:
# read in Census information by zip
zipdf = gpd.GeoDataFrame.from_file(PUIdata + '/ZIP_CODES/' + shpfile)
zipdf.head(2)

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,..."


In [11]:
df_ = pd.DataFrame()

In [12]:
for i in range(94,100):
    fname='zbp' + str(i) + 'totals.zip'
    time = 1900 + i
    zf = zipfile.ZipFile(PUIdata + '/' + fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt'))).assign(year=time)
    df.columns = [x.upper() for x in df.columns]
    df = df[['ZIP', 'EST', 'YEAR']]
    df_ = df_.append(df)

In [13]:
for i in range(0,15):
    fname='zbp' + str(i).zfill(2) + 'totals.zip'
    time = 2000 + i
    zf = zipfile.ZipFile(PUIdata + '/' + fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt'))).assign(year=time)
    df.columns = [x.upper() for x in df.columns]
    df = df[['ZIP', 'EST', 'YEAR']]
    df_ = df_.append(df)

In [14]:
zf = zipfile.ZipFile(fname)
df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
zipdf = gpd.GeoDataFrame.from_file(PUIdata + '/ZIP_CODES/' + shpfile)
zipdf.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'zbp14totals.zip'

In [ ]:
zipdf = zipdf[['ZIPCODE', 'geometry']]
zipdf['ZIPCODE'] = pd.to_numeric(zipdf['ZIPCODE'])
zipdf.head()